In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/Users/aarohiverma/Documents/ai_vs_real/output_with_glcm_features.csv')

In [6]:
df.drop(columns ='Unnamed: 0', inplace = True)

In [9]:


df = df.sample(frac=1).reset_index(drop=True)


In [17]:
df.shape

(79950, 13)

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

# Assuming df is your DataFrame with the target column 'label'
X = df.drop(columns=['label' , 'file_name'])
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Initialize and fit the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.85
Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.86      0.85     12003
           1       0.86      0.84      0.85     11982

    accuracy                           0.85     23985
   macro avg       0.85      0.85      0.85     23985
weighted avg       0.85      0.85      0.85     23985



In [24]:
df_test = pd.read_csv('/Users/aarohiverma/Documents/ai_vs_real/test_with_pred.csv')

In [25]:
df_test.head()

,id,Mean_B,Mean_G,Mean_R,StdDev_B,StdDev_G,StdDev_R,Contrast,Dissimilarity,Homogeneity,Energy,Correlation
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg,93.827995,139.799667,141.812574,66.411804,63.292838,76.193827,0.569411,0.237202,0.906969,0.229963,0.980197
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg,55.051154,66.117082,85.290761,60.792807,68.627271,77.797043,1.163743,0.541490,0.782350,0.372104,0.964919
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg,171.233147,171.233147,171.233147,46.955163,46.955163,46.955163,0.100066,0.086178,0.958126,0.324979,0.993496
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg,117.593816,137.974868,135.912936,65.250722,57.703197,63.568026,1.112489,0.674759,0.704810,0.164296,0.952797
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg,162.191591,180.409978,200.949544,88.567894,77.400920,65.158145,0.211848,0.163787,0.922640,0.433236,0.994807


In [20]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load and preprocess df_test (without modifying the ID column)
df_test_id = df_test[['id']]  # Keep the ID column separately
df_test_features = df_test.drop(columns=['id'], errors='ignore')  # Remove ID from features

# Apply the same scaler used during training
df_test_scaled = scaler.transform(df_test_features)

# Predict using the trained model
df_test_predictions = model.predict(df_test_scaled)

# Store predictions alongside their IDs
df_test_results = df_test_id.copy()
df_test_results['predictions'] = df_test_predictions

# Save to CSV (optional)
df_test_results.to_csv('pred_6_logreg.csv', index=False)

# Display sample predictions
df_test_results.head()


,id,predictions
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg,1
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg,0
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg,0
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg,0
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg,1


In [28]:


# Get probabilities instead of direct predictions
df_test_probs = model.predict_proba(df_test_scaled)[:, 1]  # Probability for class 1

# Define a custom threshold (adjust as needed)
threshold = 0.45  # Change this value to increase/decrease sensitivity

# Apply threshold to get final predictions
df_test_predictions = (df_test_probs >= threshold).astype(int)

# Store predictions alongside their IDs
df_test_results = df_test_id.copy()
df_test_results['predictions'] = df_test_predictions
df_test_results['probability'] = df_test_probs  # Optional: Store probabilities

# Save to CSV (optional)
df_test_results.to_csv('pred_6_logreg.csv', index=False)

# Display sample predictions
df_test_results.head()


,id,predictions,probability
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg,1,0.533428
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg,0,0.237378
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg,0,0.044623
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg,0,0.052505
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg,1,0.692517


In [26]:
print("Training Data Summary:")
print(df.describe())

print("\nTest Data Summary:")
print(df_test.describe())


Training Data Summary:
              label        Mean_B        Mean_G        Mean_R      StdDev_B  \
count  79950.000000  79950.000000  79950.000000  79950.000000  79950.000000   
mean       0.500000    134.983360    151.023723    159.068367     72.314200   
std        0.500003     51.281226     43.488946     46.986521     20.969937   
min        0.000000      0.786140      0.559710      1.417531      0.294121   
25%        0.000000     98.319502    122.975926    128.299856     57.736596   
50%        0.500000    136.766913    153.121001    161.284242     73.694713   
75%        1.000000    174.065058    182.519743    194.674963     88.744591   
max        1.000000    253.833737    254.886549    254.654482    124.899423   

           StdDev_G      StdDev_R      Contrast  Dissimilarity   Homogeneity  \
count  79950.000000  79950.000000  79950.000000   79950.000000  79950.000000   
mean      67.733149     68.405385      0.931104       0.361287      0.861660   
std       18.837132     2

In [29]:
import numpy as np

# Separate the ID column but don't drop it
df_test_ids = df_test["id"]  # Keep the ID column
df_test_features = df_test.drop(columns=["id"])  # Remove it from features

# Apply the same scaler used during training
df_test_scaled = scaler.transform(df_test_features)

# Get prediction probabilities
df_test_probs = model.predict_proba(df_test_scaled)[:, 1]  # Probability of class 1

# Function to test different thresholds
def find_best_threshold(probs, step=0.05):
    for t in np.arange(0.3, 0.6, step):  # Test thresholds from 0.3 to 0.6
        preds = (probs >= t).astype(int)
        print(f"Threshold {t:.2f}: {np.bincount(preds)}")

# Run threshold search
find_best_threshold(df_test_probs)



Threshold 0.30: [3639 1901]
Threshold 0.35: [3894 1646]
Threshold 0.40: [4139 1401]
Threshold 0.45: [4297 1243]
Threshold 0.50: [4420 1120]
Threshold 0.55: [4529 1011]


In [34]:
# Set final threshold (Adjust based on above output)
final_threshold = 0.15
df_test_predictions = (df_test_probs >= final_threshold).astype(int)

# Add predictions back to DataFrame with ID column
df_test_results = df_test_ids.to_frame()  # Convert ID series back to DataFrame
df_test_results["prediction"] = df_test_predictions

# Save results
df_test_results.to_csv("test_predictions_logreg.csv", index=False)
print("Predictions saved!")


Predictions saved!
